In [144]:
%matplotlib widget
import pandas as pd
import numpy as np
import geopandas
import matplotlib
matplotlib.rcParams['text.usetex'] = False
matplotlib.rcParams['text.latex.unicode'] = False
import matplotlib.pyplot as plt
from shapely.geometry import Point

# The main two projections so we can switch between them
mercator_prj = {'init': 'epsg:3395'}
platte_prj = {'init': 'epsg:4326'}

# Load the California shapefile
CA = geopandas.read_file('CA_State/CA_State_TIGER2016.shp')

# Load the reservoir information
reservoirs = pd.read_csv('../data/reservoirs.csv', index_col='Name')

# Convert the Latitude and Longitude columns to a single column of tuples
reservoirs['Coordinates'] = list(zip(reservoirs.Longitude, reservoirs.Latitude))

# apply(Point), meaning for each Coordinate run Point(that coordinate)
# effectively converting the column from a column to tuples to a column
# of Point objects (needed for the next step, by geopandas)
reservoirs['Coordinates'] = reservoirs['Coordinates'].apply(Point)

# Create the GeoDataFrame for plotting
gdf = geopandas.GeoDataFrame(reservoirs, geometry='Coordinates')

# Set the initial projection (null to begin with)
gdf.crs = platte_prj

# Re-project to Mercator
# Why? Looks better, it's what people are used to seeing (i.e. Google Maps)
gdf = gdf.to_crs(mercator_prj)

# Create the plot object
fig, axs = plt.subplots(1, 2, figsize=(15, 6))
plt.subplots_adjust(wspace=0.5)
ax = axs[0]

# Plot the California shapefile as the outline
CA.plot(ax=ax, color='white', edgecolor='black')

# Get the X,Y coordinates of the points and plot them as a scatter
# This makes it easier to index them later and check if we're hovering over them
gdf_x = pd.Series((pt.x for pt in gdf['Coordinates'].get_values()))
gdf_y = pd.Series((pt.y for pt in gdf['Coordinates'].get_values()))
sc = ax.scatter(x=gdf_x, y=gdf_y)
ax.set_title('Reservoir Sites')

# This creates the initial annotation object
# We'll later change parts of this and reuse the rest
annot = ax.annotate("", xy=(0,0), xytext=(20,20), textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->"))
annot.set_visible(False)

# Create the table
columns = ('CAP (af)', "Flow (\u03BC)", "Flow (\u03C3)", 'Lat', 'Long', 'Altitude (ft)')
rows = reservoirs.index

cell_text = []
for res in range(0, len(reservoirs)):
    reservoir = reservoirs.iloc[res]
    cell_text.append(['%1.1f' % float(x) for x in [reservoir.CAP, reservoir.FLOW_MEAN, reservoir.FLOW_STDDEV, reservoir.Latitude, reservoir.Longitude, reservoir.Elevation]])

axs[1].axis('tight')
axs[1].axis('off')

table = plt.table(cellText=cell_text,
                  rowLabels=rows,
                  colLabels=columns,
                  colWidths=[0.25,0.18,0.17,0.16,0.16,0.2],
                  loc='center')
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1, 1.5)

for col in range(0, len(columns)):
    table._cells[(0, col)].set_fontsize(10)
                    
def update_annot(ind):
    index = ind["ind"][0]
    # Get the location (in figure coordinates)
    pos = sc.get_offsets()[index]
    annot.xy = pos
    text = "{}\nElevation: {} ft".format(reservoirs.index[index], reservoirs.iloc[index]["Elevation"])
    annot.set_text(text)
    annot.get_bbox_patch().set_facecolor('blue')
    annot.get_bbox_patch().set_alpha(0.4)
    
def clear_cell_colors():
    cells = table.get_child_artists()
    for cell in cells:
        cell.set_facecolor("#ffffff")
    
def update_cell_colors(ind):
    clear_cell_colors()
    row = ind["ind"][0] + 1
    for col in range(0, len(columns)):
        table._cells[(row, col)].set_facecolor("#ff000022")
    
def hover(event):
    vis = annot.get_visible()
    if event.inaxes == ax:
        cont, ind = sc.contains(event)
        if cont:
            update_annot(ind)
            update_cell_colors(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                clear_cell_colors()
                fig.canvas.draw_idle()
                
fig.canvas.mpl_connect("motion_notify_event", hover)

# props = table.properties()
# cells = props['child_artists']
# for cell in cells:
#     print(cell)

# Show plot
plt.show()

# locs, labels = plt.xticks()
# locs_y, labels_y = plt.yticks()
# # plt.xticks(locs, [])
# print(locs, locs_y)

FigureCanvasNbAgg()